In [1]:
import os, shutil, random, glob
import cv2
import numpy as np
import pandas as pd

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# CUDA_VISIBLE_DEVICES = 2

import keras
from keras.datasets import cifar10
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

Using TensorFlow backend.


In [2]:
resize = 224
def load_data():
    imgs = os.listdir("./train/")
    num = len(imgs)
    train_data = np.empty((5000, resize, resize, 3), dtype="int32")
    train_label = np.empty((5000, ), dtype="int32")
    test_data = np.empty((5000, resize, resize, 3), dtype="int32")
    test_label = np.empty((5000, ), dtype="int32")
    for i in range(5000):
        if i % 2:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 1
        else:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 0
    for i in range(5001, 10000):
        if i % 2:
            test_data[i-5000] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-5000] = 1
        else:
            test_data[i-5000] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-5000] = 0
    return train_data, train_label, test_data, test_label

In [3]:
train_data, train_label, test_data, test_label = load_data()
train_data, test_data = train_data.astype('float32'), test_data.astype('float32')
train_data, test_data = train_data/255, test_data/255

In [4]:
# y_train = keras.utils.to_categorical(y_train, 2)
# y_val = keras.utils.to_categorical(y_val, 2)
# label = keras.utils.to_categorical(label, 2)
test_label.shape

(5000,)

In [5]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape = (resize, resize, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
model.fit(train_data, train_label,
          batch_size = 64,
          epochs = 50,
          validation_split = 0.2,
          shuffle = True)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 10s 3ms/step - loss: 0.7261 - acc: 0.5037 - val_loss: 1.3456 - val_acc: 0.5000
Epoch 2/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.7246 - acc: 0.5282 - val_loss: 0.6878 - val_acc: 0.5370
Epoch 3/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.6941 - acc: 0.5050 - val_loss: 0.6888 - val_acc: 0.5300
Epoch 4/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.7252 - acc: 0.5302 - val_loss: 0.6959 - val_acc: 0.5030
Epoch 5/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.7115 - acc: 0.5643 - val_loss: 0.6720 - val_acc: 0.6210
Epoch 6/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.6719 - acc: 0.6063 - val_loss: 0.6439 - val_acc: 0.6500
Epoch 7/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.6915 - acc: 0.6480 - val_loss: 0.9798 - val_acc: 0.5430
Epoch 8/50
40

In [7]:
scores = model.evaluate(train_data, train_label, verbose=1)
print(scores)

5000/5000 [==============================] - 4s 787us/step
[0.4199573941525072, 0.9376]


In [8]:
scores = model.evaluate(test_data, test_label, verbose=1)
print(scores)

5000/5000 [==============================] - 4s 785us/step
[1.806115913772583, 0.7442]
